In [ ]:
from pyspark.sql.functions import regexp_replace, month, year, sum, count


In [ ]:
configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": "[CLIENT_ID]",
  "fs.azure.account.oauth2.client.secret": '[SECRET_KEY]',
  "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/[TENANT_ID]/oauth2/token"
}

mount_point = "/mnt/lenskartdata"

# Unmount the directory if it is already mounted
if any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
  dbutils.fs.unmount(mount_point)

# Mount the directory
dbutils.fs.mount(
  source="abfss://lenskart-data@lenskartdataankit.dfs.core.windows.net",  # container@storageacc
  mount_point=mount_point,
  extra_configs=configs
)

/mnt/lenskartdata has been unmounted.


True

In [ ]:
%fs
ls "mnt/lenskartdata"

path,name,size,modificationTime
dbfs:/mnt/lenskartdata/raw-data/,raw-data/,0,1716552085000
dbfs:/mnt/lenskartdata/transformed-data/,transformed-data/,0,1716552098000


In [ ]:
spark

In [ ]:
customers = spark.read.format("csv").option("header","true").option("inferSchema","true").option("multiline",True).load("/mnt/lenskartdata/raw-data/customers.csv")
products = spark.read.format("csv").option("header","true").option("inferSchema","true").option("multiline",True).load("/mnt/lenskartdata/raw-data/product.csv")
stores = spark.read.format("csv").option("header","true").option("inferSchema","true").option("multiline",True).load("/mnt/lenskartdata/raw-data/store.csv")
transactions = spark.read.format("csv").option("header","true").option("inferSchema","true").option("multiline",True).load("/mnt/lenskartdata/raw-data/transaction.csv")

In [ ]:

customers=customers.withColumn("address", regexp_replace("address", "\\n", " "))
customers.show()

+-----------+----------+-----------+--------------------+----------+--------------------+--------------------+----------+
|customer_id|first name|  last name|               email|       DOB|             address|                city|    region|
+-----------+----------+-----------+--------------------+----------+--------------------+--------------------+----------+
|   CUST0001| Vardaniya|      Ghosh|vardaniyaghosh@ya...|2001-03-20|804 Deshmukh Zila...|               Alwar|    Meerut|
|   CUST0002|    Jayesh|     Bhagat|jayeshbhagat@outl...|2002-04-05|04/05, Bali Marg,...|            Chittoor|    Meerut|
|   CUST0003|   Bhamini|       Mane|bhaminimane@outlo...|1988-02-22|297, Talwar Path ...|Sangli-Miraj & Ku...|      Pune|
|   CUST0004|      Heer|  Zachariah|heerzachariah@gma...|2009-09-20|80/900 Batra Circ...|               Mango|    Mohali|
|   CUST0005|    Ranbir|    Sridhar|ranbirsridhar@gma...|2001-11-13|H.No. 558 Yohanna...|                Rewa|  Vadodara|
|   CUST0006|     Seher|

In [ ]:
customers.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- first name: string (nullable = true)
 |-- last name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- DOB: date (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)



In [ ]:
products=products.fillna(value=0)
products.show()

+------+------+------+-----------+------+-------------------+--------------------+-------------+-------------+---------+----------------+-----+
|    id|  size| width| model_name|prices|totalColorAvailable|                 sku|wishlistCount|purchaseCount|avgRating|totalNoOfRatings|  qty|
+------+------+------+-----------+------+-------------------+--------------------+-------------+-------------+---------+----------------+-----+
|146012|Medium|134 mm|  LA E13517|  1700|                  2|lenskart-air-la-e...|         2904|        81982|      4.7|             148| 3328|
|216521|Medium|139 mm|LA E15417-W|  1700|                 23|lenskart-air-la-e...|            0|         7714|      0.0|               0|  210|
|217261|Medium|139 mm|LH E16985-W|  1700|                  4|lenskart-hustlr-l...|            0|         5582|      0.0|               0| 1896|
|216712|Medium|139 mm|LA E15417-W|  1700|                 31|lenskart-air-la-e...|            0|         3887|      0.0|               0

In [ ]:
products.printSchema()

root
 |-- id: integer (nullable = true)
 |-- size: string (nullable = true)
 |-- width: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- prices: integer (nullable = true)
 |-- totalColorAvailable: integer (nullable = true)
 |-- sku: string (nullable = true)
 |-- wishlistCount: integer (nullable = true)
 |-- purchaseCount: integer (nullable = true)
 |-- avgRating: double (nullable = false)
 |-- totalNoOfRatings: integer (nullable = true)
 |-- qty: integer (nullable = true)



In [ ]:
transactions.show()

+--------+--------------+-----------+----------+--------+--------+----------+--------------+
|order_id|transaction_id|customer_id|product_id|store_id|quantity|order_date|payment_method|
+--------+--------------+-----------+----------+--------+--------+----------+--------------+
|   O0001|     TRANS0001|  CUST15390|    149409|     465|      15|2022-08-28|    Debit Card|
|   O0002|     TRANS0002|   CUST4434|    216842|    1333|       7|2022-06-28|           UPI|
|   O0003|     TRANS0003|  CUST44239|    217272|    1583|       2|2023-02-28|          Cash|
|   O0004|     TRANS0004|  CUST45287|    137713|     389|       3|2022-07-06|    Debit Card|
|   O0005|     TRANS0005|  CUST19918|    138539|    1282|       1|2022-10-22|   Credit Card|
|   O0006|     TRANS0006|  CUST48284|    149934|    1117|       1|2022-08-12|    Debit Card|
|   O0007|     TRANS0007|  CUST25562|    206463|    1924|      15|2023-04-14|   Credit Card|
|   O0008|     TRANS0008|  CUST48132|    201432|      63|      11|2023

In [ ]:
transactions.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- transaction_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- payment_method: string (nullable = true)



In [ ]:
stores.show()

+----+-----------+--------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+
|  id|lk_store_id|alt_store_name_catch|store_pickup_available|store_type_labels|        address_full|           city|         state|pin_code|
+----+-----------+--------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+
| 512|    LKST416|       Bittan Market|                  true|             COCO|Shop No E5/1, Bit...|         Bhopal|Madhya Pradesh|  462016|
| 768|   LKST1583|       Himayat Nagar|                  true|             COCO|3-6-110/1,2,3 Sho...|      Hyderabad|     Telangana|  500029|
|1024|    LKST925|          NRI layout|                  true|             COCO|No 06, Double Roa...|      Bengaluru|     Karnataka|  560016|
|1280|      ST487|         Sheoraphuli|                 false|             FOFO|178/1(149), G T R...|    Sheoraphuli|   West Bengal|  712223|
|1536|

In [ ]:
stores.printSchema()

root
 |-- id: integer (nullable = true)
 |-- lk_store_id: string (nullable = true)
 |-- alt_store_name_catch: string (nullable = true)
 |-- store_pickup_available: boolean (nullable = true)
 |-- store_type_labels: string (nullable = true)
 |-- address_full: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- pin_code: integer (nullable = true)



In [ ]:
#Find the top 10 Highest Selling Product
top_10_highest_selling_products = products.orderBy("purchaseCount", ascending = False).select("model_name","sku","purchaseCount").limit(10)
top_10_highest_selling_products.show()


+--------------------+--------------------+-------------+
|          model_name|                 sku|purchaseCount|
+--------------------+--------------------+-------------+
|Lenskart Gold Mem...|lenskart-select-m...|     10322071|
|Aqua Gold Contact...|lenskart-aquagold...|      7461575|
|Aqualens Spectacl...|aqualens-lens-len...|      7197166|
|Aqualens Spectacl...|aqualens-lens-len...|      7197166|
|Aqualens Lens Cle...|aqualens-digital-...|      4601436|
|Aqualens Lens Cle...|aqualens-digital-...|      4601436|
|Lenskart Gold MAX...|lenskart-gold-max...|      3655835|
|Lenskart Gold MAX...|lenskart-gold-max...|      3321457|
|Lenskart Gold Mem...|lenskart-gold-pre...|      3241985|
|                NULL|home-lenskart-dem...|      3042740|
+--------------------+--------------------+-------------+



In [ ]:
#Find the Lowest Selling Product
lowest_selling_product = products.orderBy("purchaseCount", ascending = True).select("model_name","sku","purchaseCount")
print(lowest_selling_product.first()[0])
print(lowest_selling_product.first()[2])

Aquacolor Combo-Mystery Brown Dusky Brown
0


In [ ]:
month_wise_sales=transactions.groupBy(month("order_date").alias("month")).agg(sum("quantity").alias("transaction_count"))
month_wise_sales.show()

+-----+-----------------+
|month|transaction_count|
+-----+-----------------+
|   12|            89742|
|    1|            88317|
|    6|            87590|
|    3|            86743|
|    5|            91459|
|    9|            86098|
|    4|            85269|
|    8|            91670|
|    7|            89847|
|   10|            87669|
|   11|            88287|
|    2|            82209|
+-----+-----------------+



In [ ]:
#Find the Highest Selling Month
months = ["January","February","March","April","May","June","July","August","September","October","November","December"]
highest_selling_month = months[month_wise_sales.orderBy("transaction_count",ascending = False).first()[0]-1]
highest_sale = month_wise_sales.orderBy("transaction_count",ascending = False).first()[1]
print(highest_selling_month)
print(highest_sale)

August
91670


In [ ]:
#Find the Lowest Selling Month
lowest_selling_month = months[month_wise_sales.orderBy("transaction_count", ascending = True).first()[0]-1]
lowest_sale = month_wise_sales.orderBy("transaction_count", ascending = True).first()[1]
print(lowest_selling_month)
print(lowest_sale)

February
82209


In [ ]:
#Find top 15 Highest Rated Product
top_15_products_by_ratings = products.orderBy("avgRating", "totalNoOfRatings", ascending = False).select("sku", "avgRating", "totalNoOfRatings").limit(15)
top_15_products_by_ratings.show()


+--------------------+---------+----------------+
|                 sku|avgRating|totalNoOfRatings|
+--------------------+---------+----------------+
|vincent-chase-ful...|      5.0|              19|
|vincent-chase-ful...|      5.0|              19|
|lenskart-reader-l...|      5.0|              17|
|lenskart-reader-l...|      5.0|              17|
|lenskart-reader-l...|      5.0|              17|
|lenskart-reader-l...|      5.0|              17|
|lenskart-gold-max...|      5.0|              15|
|hooper-hp-e10086-...|      5.0|              14|
|hooper-hp-e10086-...|      5.0|              14|
|hooper-hp-e10086-...|      5.0|              14|
|hooper-hp-e10086-...|      5.0|              14|
|hooper-hp-e10086-...|      5.0|              14|
|hooper-hp-e10086-...|      5.0|              14|
|hooper-hp-e10077-...|      5.0|              13|
|hooper-hp-e10077-...|      5.0|              13|
+--------------------+---------+----------------+



In [ ]:
#Most Wishlisted Item
products_by_wishlists = products.orderBy("wishlistCount", ascending = False).select("sku", "wishlistCount")
most_wishlisted_product = products_by_wishlists.first()[0]
most_wishlists = products_by_wishlists.first()[1]
print(most_wishlisted_product)
print(most_wishlists)

lenskart-aquagold-membership
331436


In [ ]:
#Year Wise Sales
year_wise_sales=transactions.groupBy(year("order_date").alias("year")).agg(sum("quantity").alias("Sales_count"))
year_wise_sales.show()

+----+-----------+
|year|Sales_count|
+----+-----------+
|2023|     525469|
|2022|     367994|
|2024|     161437|
+----+-----------+



In [ ]:
#Wishlist-Stock Ratio
wishlist_stock = products.select("model_name", "sku", "wishlistCount", "qty")
wishlist_stock = wishlist_stock.withColumn("wishlist/stock",wishlist_stock["wishlistCount"]/wishlist_stock["qty"])
wishlist_stock.show()

+-----------+--------------------+-------------+-----+------------------+
| model_name|                 sku|wishlistCount|  qty|    wishlist/stock|
+-----------+--------------------+-------------+-----+------------------+
|  LA E13517|lenskart-air-la-e...|         2904| 3328|0.8725961538461539|
|LA E15417-W|lenskart-air-la-e...|            0|  210|               0.0|
|LH E16985-W|lenskart-hustlr-l...|            0| 1896|               0.0|
|LA E15417-W|lenskart-air-la-e...|            0|  642|               0.0|
|  VC E13634|vincent-chase-vc-...|            0| 2285|               0.0|
|  LA E16153|lenskart-air-la-e...|            0|    3|               0.0|
|  VC E13037|vincent-chase-vc-...|         8886| 9446| 0.940715646834639|
|LA E15417-W|lenskart-air-la-e...|            0|11839|               0.0|
|LH E16985-W|lenskart-hustlr-l...|            0| 1095|               0.0|
|  VC E16665|vincent-chase-vc-...|            0|  362|               0.0|
|LH E16985-W|lenskart-hustlr-l...|    

In [ ]:
#Export from pyspark dataframe
wishlist_stock.write.mode("overwrite").option("header","true").csv("/mnt/lenskartdata/transformed-data/wishlist_stock")
products.write.mode("overwrite").option("header","true").csv("/mnt/lenskartdata/transformed-data/products")
transactions.write.mode("overwrite").option("header","true").csv("/mnt/lenskartdata/transformed-data/transactions")
customers.write.mode("overwrite").option("header","true").csv("/mnt/lenskartdata/transformed-data/customers")
stores.write.mode("overwrite").option("header","true").csv("/mnt/lenskartdata/transformed-data/stores")

In [ ]:
#Export from pandas dataframe
stores.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/stores.csv", index = False)
wishlist_stock.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/wishlist_stock.csv", index = False)
transactions.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/transactions.csv", index = False)
customers.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/customers.csv", index = False)
products.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/products.csv", index = False)
top_10_highest_selling_products.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/top_10_highest_selling_products.csv", index = False)
top_15_products_by_ratings.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/top_15_highest_rated_products.csv", index = False)

In [ ]:
#Transactions by payment methods
transactions_by_payment_methods = transactions.groupBy(transactions.payment_method.alias("payment_method")).agg(count("payment_method").alias("no_of_transactions"))
transactions_by_payment_methods.show()

+--------------+------------------+
|payment_method|no_of_transactions|
+--------------+------------------+
|   Credit Card|             24955|
|          Cash|             25036|
|    Debit Card|             25139|
|           UPI|             24870|
+--------------+------------------+



In [ ]:
#Transactions by year
transactions_by_year = transactions.groupBy(year("order_date").alias("year")).agg(count("order_id").alias("no_of_transactions"))
transactions_by_year.show()

+----+------------------+
|year|no_of_transactions|
+----+------------------+
|2023|             49993|
|2022|             34729|
|2024|             15278|
+----+------------------+



In [ ]:
transactions_by_payment_methods.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/transactions_by_payment_methods.csv", index = False)
month_wise_sales.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/month_wise_sales.csv", index = False)
transactions_by_year.toPandas().to_csv("/dbfs/mnt/lenskartdata/transformed-data/transactions_by_year.csv", index = False)